# CREDENCE -  Credence to Causal Estimates

### Premise. 
Retrospective causal inference methods, like synthetic controls or difference in difference, are widely used to estimate the impact of an intervention where A/B testing is not an option: for example, fee changes or marketing campaigns. The causal effect of an intervention is the difference between the outcome under a treatment and the counterfactual outcome if an alternative treatment (or no treatment) was chosen. The main challenge in retrospective and non-experimental causal inference is that we only observe the outcome under one of the treatment choices while the counterfactual is never observed. Causal inference methods aims to re-create the scenario in which the alternative treatment was chosen i.e. estimating the counterfactual.

### Evaluation. 
Traditional causal inference methods are typically evaluated in two ways: placebo tests and tests on simulated data. Placebo tests check that the treatment effect estimated by a particular method is zero during a period there is actually no intervention: in that situation, the actual should match the counterfactual. Testing on simulated data is limited because the data generative process is often simple and lacks the complexities of real world datasets where the method is actually applied. For applied researchers, the important question is understanding how well a method actually performs in realistic situations, not on a toy dataset.

### Framework. 
In this project, we propose and develop a framework for generating complex and realistic datasets with known treatment effects. This approach combines the best parts of placebo tests and simulation tests: the simulated dataset would have as much complexity as the real world datasets used in placebo tests, but users would be able to control the treatment effect as well as the level of endogeneity like they do in simulation tests.

Thus, generated datasets can then be used to understand the performance of different causal inference methods on various metrics, allowing scientists to choose appropriate method for a given problem. Currently, we focus on causal inference with time-series data using synthetic control method(s). 

Our framework uses a neural network based black-box data generative model called Autoregressive Variational AutoEncoder (AR-VAE), and an interpretable transformation map (ITM) to learn the distribution and sample dataset which have similar dynamics as the real datasets of interest. The AR-VAE model allows us to generate complex data while the ITM allows us to manipulate and intervene on the samples to encode treatment effects. 

Code Commit Link - https://git-codecommit.us-east-1.amazonaws.com/v1/repos/credence

### Prerequisite python libraries
Numpy, Math, Pytorch, Pytorchvision, Pytorch-lightning, Pandas, Matplotlib, Sys, OS, sbibm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add path for repo
import os
import sys
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath('../python'))
import train_arvae
import verify_causal_model

from train_arvae import exclude_small_targets, prepare_input, generate_example_sample
from train_arvae import rescale, convert_to_2d
from sbibm.metrics.mmd import mmd
from validate_arvae import c2st_new

## How to use CREDENCE

### Fetch Training Data
Read training data and store it in form (T,B,N) where T is the number of time steps, B is the number of units, N is the number of time-series in a unit.

In [ ]:
import dill
input_file = '../data/traindata_US_referral_V1_rm_zero.pkl'
# This file can be downloaded from s3://sdg-credence/data/referral_V1/processed_pkl/traindata_US_referral_V1_rm_zero.pkl
with open(input_file, 'rb') as file:
    data_input = dill.load(file)
data_input.shape

In [ ]:
B=data_input.shape[1]
N=52
T = data_input.shape[0] #length timeseries
targets = 2
adjust = 10

import dill
in_file = '../data/traindata_US_referral_V1_postprocessed_rm_zero_run2.pkl'
# This file can be downloaded from s3://sdg-credence/data/referral_V1/processed_pkl/traindata_US_referral_V1_postprocessed_rm_zero_run2.pkl
with open(in_file, 'rb') as file:
    data = dill.load(file)
data.shape

In [ ]:
data = exclude_small_targets(data = data, targets=targets)
data_norm_orig = prepare_input(data = data, targets=targets, adjust=adjust, outlier_threshold=5)

In [ ]:
data_norm, data_norm_test = train_test_split(data_norm_orig, test_size=0.2)

### Train ARVAE

##### Set Hyperparameters

In [ ]:
hyper_params = {}
hyper_params['epochs'] = 400
hyper_params['kld_weight'] = 5e-06
hyper_params['hidden_dims'] = [512,256,128]
hyper_params['latent_dim'] = 4

###### Running training function

In [ ]:
vae_model, runner = train_arvae.train(data_norm, hyper_params, 
                          output_checkpoint_path='ar_vae.ckpt')

In [ ]:
# #Load from check point
# import t_VAE
# vae_model = t_VAE.AR_VAE.load_from_checkpoint(checkpoint_path = 'ar_vae.ckpt',
#                                                       X = torch.tensor(data_norm).float(),
#                                                       latent_dim = hyper_params['latent_dim'] , 
#                                                       hidden_dims = hyper_params['hidden_dims'],
#                                                       kl_weights = hyper_params['kld_weight'],
#                                                          ).float()

In [ ]:
### to generate samples from latent space
targets_var = generate_example_sample(vae_model, targets=targets, adjust=adjust,T=T,B=B,N=N)

In [ ]:
train_arvae.plot_data(vae_model=vae_model, targets=targets, adjust=adjust, T=T, N=N)

In [ ]:
train_arvae.plot_latent_space(vae_model, data_norm_orig)

### Validation

##### Generate reference samples and algorithm samples

In [ ]:
B_test = data_norm_test.shape[0]
reference_samples = rescale(data_norm_test, targets=targets ,B=B_test, N=N, T=T, adjust=adjust)
reference_samples = convert_to_2d(reference_samples)
reference_samples = torch.tensor(reference_samples).float()
algorithm_samples = generate_example_sample(vae_model, targets=targets, B=B_test, N=N, T=T, adjust=adjust)
algorithm_samples = convert_to_2d(algorithm_samples)
algorithm_samples=torch.tensor(algorithm_samples).float()

##### Maximum Mean Discrepancy (MMD)

In [ ]:
mmd_accuracy = mmd(reference_samples, algorithm_samples)
print("MMD2: ", mmd_accuracy)

##### C2ST

In [ ]:
## c2st
#     reference_samples = torch.tensor(data_norm_orig).float()
c2st_accuracy = c2st_new(reference_samples, algorithm_samples, z_score=False, hidden_layer_sizes=(64,32))
print("C2ST: ", c2st_accuracy)